# Fine-tuning de Llama 3.2 Vision para clasificación de imágenes: Ejemplo funcional

Este cuaderno utiliza el modelo Llama 3.2 Vision y realiza un fine-tuning en un dataset público
llamado "cats_vs_dogs", disponible en Hugging Face. El objetivo es clasificar correctamente las
imágenes en dos categorías: "cats" y "dogs". También incluye una evaluación para justificar su correcto funcionamiento.

In [ ]:
!pip install unsloth transformers datasets torch torchvision matplotlib

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer
from unsloth import VisionFineTuner
from datasets import load_dataset
import torch
import matplotlib.pyplot as plt
from torchvision.transforms import Compose, Resize, ToTensor

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

In [ ]:
model_name = "llama-3.2-vision"
model = LlamaForCausalLM.from_pretrained(model_name).to(device)
tokenizer = LlamaTokenizer.from_pretrained(model_name)

El dataset "cats_vs_dogs" está disponible en Hugging Face y ya está dividido en imágenes etiquetadas.

In [ ]:
dataset = load_dataset("cats_vs_dogs", split="train[:1000]")

Convertimos las imágenes y etiquetas al formato necesario para el entrenamiento.

In [ ]:
def preprocess_image(image):
    transform = Compose([
        Resize((224, 224)),
        ToTensor(),
    ])
    return transform(image)

def preprocess(example):
    example['image'] = preprocess_image(example['image'])
    example['text'] = tokenizer(example['label'], truncation=True, padding="max_length", max_length=16)
    return example

dataset = dataset.map(preprocess)

Definimos los parámetros para el entrenamiento del modelo.

In [ ]:
finetuner = VisionFineTuner(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    learning_rate=3e-5,
    batch_size=8,
    epochs=3,
    output_dir="./llama3.2_finetuned_cats_vs_dogs"
)

Iniciamos el proceso de entrenamiento.

In [ ]:
finetuner.train()

Guardamos el modelo ajustado.

In [ ]:
finetuner.save_model("./llama3.2_finetuned_cats_vs_dogs")

Realizamos una evaluación en un subconjunto del dataset para comprobar la precisión del modelo ajustado.

In [ ]:
eval_dataset = load_dataset("cats_vs_dogs", split="train[1000:1100]").map(preprocess)
eval_results = finetuner.evaluate(eval_dataset)
print("Resultados de la evaluación:", eval_results)

Podemos visualizar algunas imágenes junto con las predicciones del modelo para verificar su rendimiento.

In [ ]:
def show_predictions(dataset, model, tokenizer):
    model.eval()
    for sample in dataset.shuffle().select(range(5)):
        image = sample['image']
        label = sample['label']
        prediction = model(image.unsqueeze(0).to(device))
        predicted_label = tokenizer.decode(prediction.argmax(dim=1))
        plt.imshow(image.permute(1, 2, 0))
        plt.title(f"Real: {label} | Predicción: {predicted_label}")
        plt.axis("off")
        plt.show()

show_predictions(eval_dataset, model, tokenizer)